In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm

from matplotlib import pyplot as plt

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_squared_error, r2_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from utils.data_processing import *
from models.logistic_regression import LogisticRegression

In [2]:
articles_df = load_article_df()
articles_df.head()

Loaded 4604 articles in df of shape (4604, 1)


,article_name
0,Áedán_mac_Gabráin
1,Åland
2,Édouard_Manet
3,Éire
4,Óengus_I_of_the_Picts


In [3]:
links_df = load_links_df(articles_df=articles_df)
links_df.head()

Loaded 119882 links in df of shape (119882, 2)
After adding missing links, there are 124492 links in df


,source,target
0,Áedán_mac_Gabráin,Bede
1,Áedán_mac_Gabráin,Columba
2,Áedán_mac_Gabráin,Dál_Riata
3,Áedán_mac_Gabráin,Great_Britain
4,Áedán_mac_Gabráin,Ireland


In [4]:
finished_df = load_finished_df()
finished_df.head()

Loaded 51318 finished paths in df of shape (51318, 7)


,hashIP,timestamp,duration,path,difficulty_rating,path_length,num_backward
0,6a3701d319fc3754,2011-02-15 03:26:49,166,"[14th_century, 15th_century, 16th_century, Pac...",NaN,9,0
1,3824310e536af032,2012-08-12 06:36:52,88,"[14th_century, Europe, Africa, Atlantic_slave_...",3.0,5,0
2,415612e93584d30e,2012-10-03 21:10:40,138,"[14th_century, Niger, Nigeria, British_Empire,...",NaN,8,0
3,64dd5cd342e3780c,2010-02-08 07:25:25,37,"[14th_century, Renaissance, Ancient_Greece, Gr...",NaN,4,0
4,015245d773376aab,2013-04-23 15:27:08,175,"[14th_century, Italy, Roman_Catholic_Church, H...",3.0,7,0


In [5]:
unifinished_df = load_unfinished_df()
unifinished_df.head()

Loaded 24875 unfinished paths in df of shape (24875, 8)


,hashIP,timestamp,duration,path,target_article,type_end,path_length,num_backward
0,2426091a53125110,2011-02-07 05:02:15,1804,[Obi-Wan_Kenobi],Microsoft,timeout,1,0
1,26141fd878806294,2011-02-07 05:14:11,1805,[Julius_Caesar],Caracas,timeout,1,0
2,2b015fb8181c48f2,2011-02-07 15:00:19,1818,"[Malawi, Democracy, Alexander_the_Great]",First_Crusade,timeout,3,0
3,53a53bc244e08a6a,2011-02-07 16:06:01,49,[Paraguay],Mount_St._Helens,restart,1,0
4,53a53bc244e08a6a,2011-02-07 17:18:25,1808,"[Paraguay, Bolivia]",Mount_St._Helens,timeout,2,0


In [6]:
all_games_df, _ = preprocess_and_concat_unfinished_and_finished(unifinished_df, finished_df)
all_games_df.head()

After filtering all paths after 2011-02-07 05:02:15
we kept 23245 paths out of 51318 finished paths
There are 24875 unfinished paths


,difficulty_rating,duration,finished,hashIP,num_backward,path,path_length,source,target,timestamp,type_end
0,NaN,166,True,6a3701d319fc3754,0,"[14th_century, 15th_century, 16th_century, Pac...",9,14th_century,African_slave_trade,2011-02-15 03:26:49,NaN
1,3.0,88,True,3824310e536af032,0,"[14th_century, Europe, Africa, Atlantic_slave_...",5,14th_century,African_slave_trade,2012-08-12 06:36:52,NaN
2,NaN,138,True,415612e93584d30e,0,"[14th_century, Niger, Nigeria, British_Empire,...",8,14th_century,African_slave_trade,2012-10-03 21:10:40,NaN
3,3.0,175,True,015245d773376aab,0,"[14th_century, Italy, Roman_Catholic_Church, H...",7,14th_century,John_F._Kennedy,2013-04-23 15:27:08,NaN
4,NaN,110,True,5295bca242be81fe,0,"[14th_century, Europe, North_America, United_S...",6,14th_century,John_F._Kennedy,2013-07-03 22:26:54,NaN


In [7]:
valid_games_df = prune_invalid_games(all_games_df, articles_df)
valid_games_df.head()

Pruning invalid games. Initially we have 48120 games
Pruned invalid games. Now we have 48092 valid games


,difficulty_rating,duration,finished,hashIP,num_backward,path,path_length,source,target,timestamp,type_end
0,NaN,166,True,6a3701d319fc3754,0,"[14th_century, 15th_century, 16th_century, Pac...",9,14th_century,African_slave_trade,2011-02-15 03:26:49,NaN
1,3.0,88,True,3824310e536af032,0,"[14th_century, Europe, Africa, Atlantic_slave_...",5,14th_century,African_slave_trade,2012-08-12 06:36:52,NaN
2,NaN,138,True,415612e93584d30e,0,"[14th_century, Niger, Nigeria, British_Empire,...",8,14th_century,African_slave_trade,2012-10-03 21:10:40,NaN
3,3.0,175,True,015245d773376aab,0,"[14th_century, Italy, Roman_Catholic_Church, H...",7,14th_century,John_F._Kennedy,2013-04-23 15:27:08,NaN
4,NaN,110,True,5295bca242be81fe,0,"[14th_century, Europe, North_America, United_S...",6,14th_century,John_F._Kennedy,2013-07-03 22:26:54,NaN


In [8]:
_ = dump_adjacency_list(links_df, articles_df)

Dumped adjacency list with 4604 articles and 124492 entries


In [9]:
_ = dump_unique_source_target_pairs(valid_games_df)

Dumped 29834 unique source-target pairs


In [10]:
unique_game_stats_df = load_or_compute_unique_source_target_pair_stats()
unique_game_stats_df.head()

Loaded 29834 unique source-target pair stats


shortest_path_length  \
source       target                                         
14th_century African_slave_trade                        3   
             John_F._Kennedy                            3   
             Rainbow                                    3   
             Sodium                                     3   
             Elizabeth_I_of_England                     2   

                                     shortest_path_count  max_sp_pagerank  \
source       target                                                         
14th_century African_slave_trade                       3         0.001523   
             John_F._Kennedy                          18         0.007202   
             Rainbow                                   5         0.003356   
             Sodium                                    4         0.002184   
             Elizabeth_I_of_England                    1         0.000644   

                                     max_sp_avg_pagerank  avg_sp_avg_pagerank  \
source       target                                                             
14th_century African_slave_trade                0.000559             0.000414   
             John_F._Kennedy                    0.003170             0.001738   
             Rainbow                            0.000990             0.000457   
             Sodium                             0.000765             0.000519   
             Elizabeth_I_of_England             0.000371             0.000371   

                                     one_longer_path_count  max_ol_pagerank  \
source       target                                                           
14th_century African_slave_trade                        85         0.007202   
             John_F._Kennedy                          1758         0.007202   
             Rainbow                                   198         0.005009   
             Sodium                                    548         0.007202   
             Elizabeth_I_of_England                     61         0.005009   

                                     max_ol_avg_pagerank  avg_ol_avg_pagerank  \
source       target                                                             
14th_century African_slave_trade                0.002545             0.000857   
             John_F._Kennedy                    0.003519             0.001507   
             Rainbow                            0.001775             0.000671   
             Sodium                             0.002669             0.000756   
             Elizabeth_I_of_England             0.001590             0.000683   

                                     two_longer_pagerank  max_tl_pagerank  \
source       target                                                         
14th_century African_slave_trade                     300         0.005009   
             John_F._Kennedy                         300         0.007202   
             Rainbow                                 300         0.005009   
             Sodium                                  300         0.005009   
             Elizabeth_I_of_England                  300         0.005009   

                                     max_tl_avg_pagerank  
source       target                                       
14th_century African_slave_trade                0.001814  
             John_F._Kennedy                    0.003492  
             Rainbow                            0.002008  
             Sodium                             0.002164  
             Elizabeth_I_of_England             0.001854

In [11]:
node_stats_df = load_or_compute_node_stats()
node_stats_df.head()

Loaded 4604 node stats


,degree,closeness,betweenness,pagerank
article_name,,,,
10th_century,27,0.323966,5575.09,0.000500
11th_century,49,0.336178,6653.51,0.000435
12th_century,46,0.338251,14201.00,0.000561
13th_century,35,0.321630,8544.39,0.000586
14th_century,32,0.325238,5366.38,0.000470


In [12]:
valid_games_df = valid_games_df.merge(unique_game_stats_df, on=['source', 'target'], how='left')
valid_games_df.dropna(subset=['shortest_path_length'], inplace=True)
print("Number of valid games after shortest path length filtering: ", len(valid_games_df))

Number of valid games after shortest path length filtering:  48092


In [13]:
embeddings_df = load_embeddings()
embeddings_df.head()

Loaded 4604 embeddings in df of shape (4604, 1)


,embedding
article_name,
10th_century,"[-0.050105415, 0.09074813, 0.008811204, -0.051..."
11th_century,"[-0.042490385, 0.061723217, -0.020333063, -0.0..."
12th_century,"[-0.032364372, 0.037580367, -0.052070234, -0.1..."
13th_century,"[-0.07914139, 0.059444286, 0.020845776, -0.100..."
14th_century,"[-0.040906686, 0.122400954, -0.04961793, 0.007..."


In [14]:
valid_games_df = compute_cosine_similarity(valid_games_df, embeddings_df)


In [15]:
valid_games_df = merge_with_node_data(valid_games_df, node_stats_df)
valid_games_df.head()

,difficulty_rating,duration,finished,hashIP,num_backward,path,path_length,source,target,timestamp,...,max_tl_avg_pagerank,cosine_sim_source_target,degree_source,closeness_source,betweenness_source,pagerank_source,degree_target,closeness_target,betweenness_target,pagerank_target
0,NaN,166,True,6a3701d319fc3754,0,"[14th_century, 15th_century, 16th_century, Pac...",9,14th_century,African_slave_trade,2011-02-15 03:26:49,...,0.001814,0.202444,32,0.325238,5366.38,0.00047,25,0.335511,102.483,0.000028
1,3.0,88,True,3824310e536af032,0,"[14th_century, Europe, Africa, Atlantic_slave_...",5,14th_century,African_slave_trade,2012-08-12 06:36:52,...,0.001814,0.202444,32,0.325238,5366.38,0.00047,25,0.335511,102.483,0.000028
2,NaN,138,True,415612e93584d30e,0,"[14th_century, Niger, Nigeria, British_Empire,...",8,14th_century,African_slave_trade,2012-10-03 21:10:40,...,0.001814,0.202444,32,0.325238,5366.38,0.00047,25,0.335511,102.483,0.000028
3,3.0,175,True,015245d773376aab,0,"[14th_century, Italy, Roman_Catholic_Church, H...",7,14th_century,John_F._Kennedy,2013-04-23 15:27:08,...,0.003492,0.079502,32,0.325238,5366.38,0.00047,69,0.348326,16585.800,0.000243
4,NaN,110,True,5295bca242be81fe,0,"[14th_century, Europe, North_America, United_S...",6,14th_century,John_F._Kennedy,2013-07-03 22:26:54,...,0.003492,0.079502,32,0.325238,5366.38,0.00047,69,0.348326,16585.800,0.000243


In [16]:
valid_games_df.columns

Index(['difficulty_rating', 'duration', 'finished', 'hashIP', 'num_backward',
       'path', 'path_length', 'source', 'target', 'timestamp', 'type_end',
       'shortest_path_length', 'shortest_path_count', 'max_sp_pagerank',
       'max_sp_avg_pagerank', 'avg_sp_avg_pagerank', 'one_longer_path_count',
       'max_ol_pagerank', 'max_ol_avg_pagerank', 'avg_ol_avg_pagerank',
       'two_longer_pagerank', 'max_tl_pagerank', 'max_tl_avg_pagerank',
       'cosine_sim_source_target', 'degree_source', 'closeness_source',
       'betweenness_source', 'pagerank_source', 'degree_target',
       'closeness_target', 'betweenness_target', 'pagerank_target'],
      dtype='object')

In [17]:
valid_games_df = valid_games_df[valid_games_df['type_end'] != 'timeout']

In [18]:
fame_df = load_fame()
fame_df[fame_df.isna().any(axis=1)]
#fame_df.head()

,fame_score
article_name,


In [19]:
valid_games_df = merge_with_fame_data(valid_games_df, fame_df)
valid_games_df.head()

,difficulty_rating,duration,finished,hashIP,num_backward,path,path_length,source,target,timestamp,...,degree_source,closeness_source,betweenness_source,pagerank_source,degree_target,closeness_target,betweenness_target,pagerank_target,fame_score_source,fame_score_target
0,NaN,166,True,6a3701d319fc3754,0,"[14th_century, 15th_century, 16th_century, Pac...",9,14th_century,African_slave_trade,2011-02-15 03:26:49,...,32,0.325238,5366.38,0.00047,25,0.335511,102.483,0.000028,8.0,6.0
1,3.0,88,True,3824310e536af032,0,"[14th_century, Europe, Africa, Atlantic_slave_...",5,14th_century,African_slave_trade,2012-08-12 06:36:52,...,32,0.325238,5366.38,0.00047,25,0.335511,102.483,0.000028,8.0,6.0
2,NaN,138,True,415612e93584d30e,0,"[14th_century, Niger, Nigeria, British_Empire,...",8,14th_century,African_slave_trade,2012-10-03 21:10:40,...,32,0.325238,5366.38,0.00047,25,0.335511,102.483,0.000028,8.0,6.0
3,3.0,175,True,015245d773376aab,0,"[14th_century, Italy, Roman_Catholic_Church, H...",7,14th_century,John_F._Kennedy,2013-04-23 15:27:08,...,32,0.325238,5366.38,0.00047,69,0.348326,16585.800,0.000243,8.0,9.0
4,NaN,110,True,5295bca242be81fe,0,"[14th_century, Europe, North_America, United_S...",6,14th_century,John_F._Kennedy,2013-07-03 22:26:54,...,32,0.325238,5366.38,0.00047,69,0.348326,16585.800,0.000243,8.0,9.0


In [20]:
# Drop some variables before logistic regression
valid_games_df.drop(
    columns=[
        'hashIP', 'timestamp', 'path', 'difficulty_rating', 'type_end',
        'duration', 'path_length', 'num_backward'
    ], 
    inplace=True
)

valid_games_df = valid_games_df[valid_games_df['shortest_path_length'] > 0]

# The number of paths is better when log-transformed
valid_games_df['shortest_path_count'] = valid_games_df['shortest_path_count'].apply(lambda x: np.log(x + 1e-2))
valid_games_df['one_longer_path_count'] = valid_games_df['one_longer_path_count'].apply(lambda x: np.log(x + 1e-2))

# Change the type of finished to a float for logistic regression
valid_games_df['finished'] = valid_games_df['finished'].astype(float)

print(valid_games_df.shape)
print(valid_games_df.columns)

(38762, 26)
Index(['finished', 'source', 'target', 'shortest_path_length',
       'shortest_path_count', 'max_sp_pagerank', 'max_sp_avg_pagerank',
       'avg_sp_avg_pagerank', 'one_longer_path_count', 'max_ol_pagerank',
       'max_ol_avg_pagerank', 'avg_ol_avg_pagerank', 'two_longer_pagerank',
       'max_tl_pagerank', 'max_tl_avg_pagerank', 'cosine_sim_source_target',
       'degree_source', 'closeness_source', 'betweenness_source',
       'pagerank_source', 'degree_target', 'closeness_target',
       'betweenness_target', 'pagerank_target', 'fame_score_source',
       'fame_score_target'],
      dtype='object')


In [21]:
features_1 = ['shortest_path_length', 'shortest_path_count']

model_1 = LogisticRegression(valid_games_df, features_1)
model_1.fit()

Class distribution: finished
0.0    0.5
1.0    0.5
Name: proportion, dtype: float64
Total number of samples: 31052


KeyError: 'n'

In [ ]:
model_1.summary()

In [ ]:
features_2 = ['pagerank_target']

model_2 = LogisticRegression(valid_games_df, features_2, has_constant_term=True)
model_2.fit()

In [ ]:
model_2.summary()

In [ ]:
features_3 = [
    'shortest_path_length',
    'shortest_path_count', 'max_sp_pagerank', 'max_sp_avg_pagerank',
    'avg_sp_avg_pagerank', 'one_longer_path_count', 'max_ol_pagerank',
    'max_ol_avg_pagerank', 'avg_ol_avg_pagerank', 'two_longer_pagerank',
    'max_tl_pagerank', 'max_tl_avg_pagerank', 'cosine_similarity',
    'degree_source', 'closeness_source', 'betweenness_source',
    'pagerank_source', 'degree_target', 'closeness_target',
    'betweenness_target', 'pagerank_target', 'fame_score_source', 'fame_score_target'
]

model_3 = LogisticRegression(valid_games_df, features_3)
model_3.fit()

In [ ]:
model_3.summary()

In [ ]:
combined_pred = valid_games_df.copy()[['finished']]

combined_pred['model_1_pred'] = model_1.predict(valid_games_df[features_1])
combined_pred['model_2_pred'] = model_2.predict(valid_games_df[features_2])
combined_pred['model_3_pred'] = model_3.predict(valid_games_df[features_3])

combined_pred['pred'] = combined_pred[['model_1_pred', 'model_2_pred', 'model_3_pred']].sum(axis=1)
combined_pred['pred'] = combined_pred['pred'].apply(lambda x: 1 if x >= 2 else 0)

print(classification_report(combined_pred['finished'], combined_pred['pred'], digits=4))